# <font color=yellow> Static Data Synchronisation </font>


The Static Data Synchronisation sample outlines the process of seamlessly integrating and synchronizing core data between Kyriba and your system. By leveraging Kyriba's APIs and webhooks, you can establish a reliable connection to facilitate the exchange of static data, ensuring accuracy and consistency across both platforms.<br>

For demonstrative purposes, this sample utilizes the "Third party" data type.<br>
The following actions are showcased:
*   Retrieving a list of all third parties.
*   Retrieving details of a single third party.
*   Creating, Updating, and Deleting a third party.
*   Dispatching a Webhook notification once a third party is changed - created, updated, or deleted.


## <font color='yellow'>Code for token generation and API management</font>
      

In [ ]:
#@title Select **servicePack** the Sample should run
from subprocess import getstatusoutput
servicePack = "DEMO" #@param ['DEMO'] {allow-input: true}

if servicePack == 'DEMO':
  branch = 'main'
else:
  branch = servicePack
print(branch)
rm = getstatusoutput("rm Colab_shared_code_for_API_usage -rf ")
clone = getstatusoutput("git clone -l -s --branch " +  branch + " https://github.com/kyriba/Colab_shared_code_for_API_usage.git")
import importlib
try:
  importlib.reload(sample_requests)
except:
  print()
from Colab_shared_code_for_API_usage import sample_requests


In [ ]:
#@title Import custom modules
import time
from datetime import datetime,date
import json as js
import copy
import numpy as np


In [ ]:
#@title Additional functions
def show_results(df):
  df2 = copy.deepcopy(df)
  df2.replace(np.nan, '-', inplace = True)
  display(df2)

## <font color='yellow'>Connect and  retrieve token</font>

In [ ]:
#@title <font color='lime'>Connect</font>
print('token:')
token = sample_requests.login()

## <font color='yellow'>Third Party provisioning</font>

In [ ]:
#@title <font color='lime'>Retrieve all third parties</font>
try:
  # get all third parties in json
  res2 = sample_requests.get_results(token, '/v1/third-parties?page.limit=1000')
  ppjson = js.loads(res2, parse_int=int)
  # list of columns to display results from Third Parties
  colunmsTP = ['uuid', 'code', 'name', 'creationDate', 'updateDate', 'activeStatus']
  # convert json to dataFrame
  df2 = sample_requests.pd.json_normalize(ppjson['results']).filter(items=['uuid', 'code', 'name', 'creationDate', 'updateDate', 'activeStatus', 'businessIdentifiers.otherIdentifierType', 'address.country.code'], axis =1)
  df2 = df2[colunmsTP]
  show_results(df2)
except Exception as e:
  print('No Third party was found.')

In [ ]:
#@title <font color='lime'>Get details about third party by reference</font>
ref = '4-PARTY-TEST' #@param {type:"string"}
#@markdown ****ref***  is the reference (uuid or code) of the third party.
# get all third parties in json
try:
  res2 = sample_requests.get_results(token, '/v1/third-parties/' + ref)
  ppjson = js.loads(res2, parse_int=int)
  # list of columns to display results from Third Parties
  colunmsTP = ['uuid', 'code', 'name', 'hidden', 'creditor', 'debtor', 'nonResident', 'transactionEntryLimit', 'transactionMaxNumber', 'companySelection', 'limitCurrency.code', 'address.country.code']
  # convert json to dataFrame
  df2 = sample_requests.pd.json_normalize(ppjson)
  df2 = df2[colunmsTP]
  show_results(df2)
except:
  print()


In [ ]:
#@title <font color='lime'> Create a third party</font>
#@markdown Provide values for parameters below:
code = '3-PARTY-TEST' #@param {type:"string"}
addressCountryCode = 'GB' #@param ['FR', 'GB', 'GG', 'JP', 'PL', 'SG', 'US'] {allow-input: true}
name = 'Name' #@param {type: "string"}

checker = 'false'

payload = sample_requests.js.dumps({
  "code": code,
  "address": {
    "country": {
      "code": addressCountryCode
    }
  },
  "name": name
})

#print(payload)

try:
  res2 = sample_requests.post_results(token, '/v1/third-parties', payload, {}, {'Content-Type': 'application/json'})
  print(res2)
  try:
    if res2['code'] == code:
      checker = 'true'
      print('\nThird party ' + code + ' was succesfully created in Kyriba.')
      print()
      # print(res2)
  except:
    print()
    # print('Error')
    print("\nThird party " + code + " could not be created in Kyriba.")
except sample_requests.ResponceError as err:
  print('Third party ' + code + ' could not be created in Kyriba.')
print()
print()

In [ ]:
#@title <font color='lime'> Update a third party</font>
#@markdown Provide values for parameters below:
ref = '3-PARTY-TEST' #@param {type:"string"}
name = 'New Name' #@param {type: "string"}
addressCountryCode = 'FR' #@param ['FR', 'GB', 'GG', 'JP', 'PL', 'SG', 'US'] {allow-input: true}
checker = 'false'
#@markdown ****ref***  is the reference (uuid or code) of the third party.
try:
  third_party_to_update = sample_requests.get_results(token, '/v1/third-parties/' + ref)
  third_party_to_update = js.loads(third_party_to_update)
  # print(sample_requests.pp_json(third_party_to_update))
  third_party_to_update['name'] = name
  third_party_to_update['address']['country']['code'] = addressCountryCode
  third_party_to_update.pop('internalCounter')
  third_party_to_update.pop('internalCounterSuffix')
  third_party_to_update['address']['country'].pop('uuid')
  updated_third_party = sample_requests.put_results(token, '/v1/third-parties/' + ref, sample_requests.js.dumps(third_party_to_update), {}, {'Content-Type': 'application/json'})
  if updated_third_party['uuid'] == ref or updated_third_party['code'] == ref:
      checker = 'true'
      print('\nThird party ' + ref + ' was succesfully updated in Kyriba.')
      print()
      print(updated_third_party)
except Exception as err:
  print('Third party ' + ref + ' could not be updated.')
print()
print()

In [ ]:
#@title <font color='lime'> Delete a third party</font>
#@markdown Provide the reference of third party to delete:
ref = '3-PARTY-TEST' #@param {type:"string"}
#@markdown ****ref***  is the reference (uuid or code) of the third party.
checker = 'false'

try:
  deleted_third_party = sample_requests.delete_results(token, '/v1/third-parties/' + ref, {'Content-Type': 'application/json'})
  try:
    if deleted_third_party['uuid'] == ref or deleted_third_party['code'] == ref:
      checker = 'true'
      print('\nThird party ' + ref + ' was succesfully deleted in Kyriba.')
      print()
      print(deleted_third_party)
  except:
    print()
    # print('Error')
    print("\nThird party " + ref + " could not be deleted.")
except Exception as err:
  print('Third party ' + ref + ' could not be deleted.')
print()
print()

## <font color='yellow'>Third Party synchronisation</font>
There are two ways to synchronise core data:
* Configure Webhook notifications (if available for a given data type) that are sent by Kyriba once a change has occured.
* Poll Kyriba APIs utilizing creation or update date filters.


In [ ]:
#@title <font color='lime'> Run Webhook dashboard <font>

#@markdown With Webhooks, you will receive a notification for any change (creation, modification, deletion) of your third parties.

#@markdown Run the block to display the Webhook dashboard which demonstrates how the Webhook notifications look like.
#@markdown This is the place where notifications are sent for the purpose of this sample.\
#@markdown Its left side - the "Messages" section - represents a list of received notifications.
#@markdown Once a notification from the list is selected, the middle part of the dashboard will display the notification details. Go to the "Request Body" section to see the API request received.

#@markdown #![Screenshot 2023-09-08 at 17.50.56.png](...)

from IPython.display import IFrame

IFrame(src='https://play.svix.com/view/e_wdUPMbdZ1RpJnt0FaltbHlqxXCI/', width=1000, height=600)

In [ ]:
#@title <font color='lime'>Show created and updated third parties</font>
creationDate = '2023-05-11' #@param {type:"date"}
creationFilter = 'Equals' #@param ['Equals', 'Greather than', 'Greater Equal than'] {type: "string"}
updateDate = '2023-05-11' #@param {type:"date"}
updateFilter = 'Equals' #@param ['Equals', 'Greather than', 'Greater Equal than'] {type: "string"}
#@markdown
showResults = 'Only created' #@param ['Only created', 'All modified', 'Created and modified']

if updateFilter == 'Equals':
  updateFilter = '='
elif updateFilter == 'Greather than':
  updateFilter = '=gt'
elif updateFilter == 'Greater Equal than':
  updateFilter = '=ge'

if creationFilter == 'Equals':
  creationFilter = '='
elif creationFilter == 'Greather than':
  creationFilter = '=gt'
elif creationFilter == 'Greater Equal than':
  creationFilter = '=ge'

create_filter = 'creationDate' + creationFilter + '=' + creationDate
update_filter = 'updateDate' + updateFilter + '=' + updateDate
if showResults == 'Only created':
  filter = create_filter + ';updateDate==null'
elif showResults == 'All modified':
  filter = update_filter
else:
  filter = create_filter + ',' + update_filter
# print(filter)
try:
  # get all third parties in json
  res2 = sample_requests.get_results(token, '/v1/third-parties?filter=' + filter)
  ppjson = js.loads(res2, parse_int=int)
  # list of columns to display results from Third Parties
  colunmsTP = ['uuid', 'code', 'name', 'creationDate', 'updateDate', 'activeStatus']
  # convert json to dataFrame
  df2 = sample_requests.pd.json_normalize(ppjson['results']).filter(items=['uuid', 'code', 'name', 'creationDate', 'updateDate', 'activeStatus', 'businessIdentifiers.otherIdentifierType', 'address.country.code'], axis =1)
  df2 = df2[colunmsTP]

  show_results(df2)
except Exception as e:
  print('No Third party with applied filter was found.')



In [ ]:
#@title <font color='lime'> Show deleted third parties</font>
#@markdown There isn't an endpoint to retrieve deleted third parties. To detect those deleted ones, perform LIST without filter and compare the total result with the ones you have. If there is a mismatch, then look for the difference in uuids.